# Description

(Please, take a look at the README.md file in this directory for instructions on how to run this notebook)

**TODO:** update

This notebook computes predicted expression correlations between all genes in the MultiPLIER models.

It also has a parameter set for papermill to run on a single chromosome to run in parallel (see under `Settings` below).

This notebook is not directly run. See README.md.

# Modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pickle

import numpy as np

import pandas as pd
from tqdm import tqdm

import conf
from entity import Gene

# Settings

In [3]:
# a cohort name (it could be something like UK_BIOBANK, etc)
COHORT_NAME = None

# a string with a path pointing to an imputed GWAS
GWAS_FILE = None

# a string with a path pointing where S-PrediXcan results (tissue-specific are located
SPREDIXCAN_FOLDER = None

# an f-string with one placeholder {tissue}
SPREDIXCAN_FILE_PATTERN = None

# a string with a path pointing to an S-MultiXcan result
SMULTIXCAN_FILE = None

# predictions models such as MASHR or ELASTIC_NET
EQTL_MODEL = None

In [4]:
# Parameters
COHORT_NAME = "1000G_EUR"
GWAS_FILE = "/opt/data/results/gls/null_sims/final_imputed_gwas/random.pheno0.glm-imputed.txt.gz"
SPREDIXCAN_FOLDER = "/opt/data/results/gls/null_sims/twas/spredixcan/"
SPREDIXCAN_FILE_PATTERN = "random.pheno0-gtex_v8-mashr-{tissue}.csv"
SMULTIXCAN_FILE = "/opt/data/results/gls/null_sims/twas/smultixcan/random.pheno0-gtex_v8-mashr-smultixcan.txt"
EQTL_MODEL = "MASHR"


In [5]:
assert COHORT_NAME is not None and len(COHORT_NAME) > 0, "A cohort name must be given"

COHORT_NAME = COHORT_NAME.lower()
display(f"Cohort name: {COHORT_NAME}")

'Cohort name: 1000g_eur'

In [6]:
assert GWAS_FILE is not None and len(GWAS_FILE) > 0, "A GWAS file path must be given"
GWAS_FILE = Path(GWAS_FILE).resolve()
assert GWAS_FILE.exists(), "GWAS file does not exist"

display(f"GWAS file path: {str(GWAS_FILE)}")

'GWAS file path: /opt/data/results/gls/null_sims/final_imputed_gwas/random.pheno0.glm-imputed.txt.gz'

In [7]:
assert (
    SPREDIXCAN_FOLDER is not None and len(SPREDIXCAN_FOLDER) > 0
), "An S-PrediXcan folder path must be given"
SPREDIXCAN_FOLDER = Path(SPREDIXCAN_FOLDER).resolve()
assert SPREDIXCAN_FOLDER.exists(), "S-PrediXcan folder does not exist"

display(f"S-PrediXcan folder path: {str(SPREDIXCAN_FOLDER)}")

'S-PrediXcan folder path: /opt/data/results/gls/null_sims/twas/spredixcan'

In [8]:
assert (
    SPREDIXCAN_FILE_PATTERN is not None and len(SPREDIXCAN_FILE_PATTERN) > 0
), "An S-PrediXcan file pattern must be given"
assert (
    "{tissue}" in SPREDIXCAN_FILE_PATTERN
), "S-PrediXcan file pattern must have a '{tissue}' placeholder"

display(f"S-PrediXcan file template: {SPREDIXCAN_FILE_PATTERN}")

'S-PrediXcan file template: random.pheno0-gtex_v8-mashr-{tissue}.csv'

In [9]:
assert (
    SMULTIXCAN_FILE is not None and len(SMULTIXCAN_FILE) > 0
), "An S-MultiXcan result file path must be given"
SMULTIXCAN_FILE = Path(SMULTIXCAN_FILE).resolve()
assert SMULTIXCAN_FILE.exists(), "S-MultiXcan result file does not exist"

display(f"S-MultiXcan file path: {str(SMULTIXCAN_FILE)}")

'S-MultiXcan file path: /opt/data/results/gls/null_sims/twas/smultixcan/random.pheno0-gtex_v8-mashr-smultixcan.txt'

In [10]:
assert (
    EQTL_MODEL is not None and len(EQTL_MODEL) > 0
), "A prediction/eQTL model must be given"

display(f"eQTL model: {EQTL_MODEL}")

'eQTL model: MASHR'

In [11]:
OUTPUT_DIR_BASE = conf.RESULTS["GLS"] / "gene_corrs" / "cohorts" / COHORT_NAME
OUTPUT_DIR_BASE.mkdir(parents=True, exist_ok=True)

display(f"Using output dir base: {OUTPUT_DIR_BASE}")

'Using output dir base: /opt/data/results/gls/gene_corrs/cohorts/1000g_eur'

# Load MultiPLIER Z genes

In [12]:
multiplier_z_genes = pd.read_pickle(
    conf.MULTIPLIER["MODEL_Z_MATRIX_FILE"]
).index.tolist()

In [13]:
len(multiplier_z_genes)

6750

In [14]:
assert len(multiplier_z_genes) == len(set(multiplier_z_genes))

In [15]:
multiplier_z_genes[:5]

['GAS6', 'MMP14', 'DSP', 'MARCKSL1', 'SPARC']

# GWAS

In [16]:
gwas_file_columns = pd.read_csv(GWAS_FILE, sep="\t", nrows=2).columns
assert (
    "panel_variant_id" in gwas_file_columns
), "GWAS file must be final imputed one with column 'panel_variant_id'"
# FIXME: add other needed columns here

In [17]:
gwas_data = pd.read_csv(
    GWAS_FILE,
    sep="\t",
    usecols=["panel_variant_id", "pvalue", "zscore", "imputation_status"],
)

In [18]:
gwas_data.shape

(8339505, 4)

In [19]:
gwas_data.head()

,panel_variant_id,zscore,pvalue,imputation_status
0,chr1_54490_G_A_b38,1.567452,0.117009,original
1,chr1_87021_T_C_b38,0.244372,0.806943,imputed
2,chr1_263722_C_G_b38,1.155239,0.247993,imputed
3,chr1_594402_C_T_b38,0.851234,0.394639,imputed
4,chr1_630555_C_T_b38,0.855129,0.392480,imputed


In [20]:
gwas_data["imputation_status"].unique()

array(['original', 'imputed'], dtype=object)

In [21]:
gwas_data.dropna().shape

(8339505, 4)

In [22]:
# remove SNPs with no results
gwas_data = gwas_data.dropna()

In [23]:
gwas_data.shape

(8339505, 4)

## Save GWAS variants

In [24]:
gwas_data.head()

,panel_variant_id,zscore,pvalue,imputation_status
0,chr1_54490_G_A_b38,1.567452,0.117009,original
1,chr1_87021_T_C_b38,0.244372,0.806943,imputed
2,chr1_263722_C_G_b38,1.155239,0.247993,imputed
3,chr1_594402_C_T_b38,0.851234,0.394639,imputed
4,chr1_630555_C_T_b38,0.855129,0.392480,imputed


In [25]:
assert gwas_data["panel_variant_id"].is_unique

In [26]:
gwas_variants_ids_set = frozenset(gwas_data["panel_variant_id"])
list(gwas_variants_ids_set)[:5]

['chr2_165831678_T_C_b38',
 'chr4_85678484_A_C_b38',
 'chr13_18893534_A_T_b38',
 'chr13_30710910_A_G_b38',
 'chr1_233708632_G_C_b38']

In [27]:
with open(OUTPUT_DIR_BASE / "gwas_variant_ids.pkl", "wb") as handle:
    pickle.dump(gwas_variants_ids_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

# TWAS

## Available tissues for eQTL model

In [28]:
prediction_model_tissues = conf.PHENOMEXCAN["PREDICTION_MODELS"][
    f"{EQTL_MODEL}_TISSUES"
].split(" ")

In [29]:
len(prediction_model_tissues)

49

In [30]:
prediction_model_tissues[:5]

['Skin_Not_Sun_Exposed_Suprapubic',
 'Cells_EBV-transformed_lymphocytes',
 'Brain_Frontal_Cortex_BA9',
 'Kidney_Cortex',
 'Brain_Substantia_nigra']

## S-PrediXcan results

### Load results across all tissues

In [31]:
spredixcan_result_files = {
    t: SPREDIXCAN_FOLDER / SPREDIXCAN_FILE_PATTERN.format(tissue=t)
    for t in prediction_model_tissues
}

In [32]:
assert len(spredixcan_result_files) == len(prediction_model_tissues)
display(list(spredixcan_result_files.values())[:5])

[PosixPath('/opt/data/results/gls/null_sims/twas/spredixcan/random.pheno0-gtex_v8-mashr-Skin_Not_Sun_Exposed_Suprapubic.csv'),
 PosixPath('/opt/data/results/gls/null_sims/twas/spredixcan/random.pheno0-gtex_v8-mashr-Cells_EBV-transformed_lymphocytes.csv'),
 PosixPath('/opt/data/results/gls/null_sims/twas/spredixcan/random.pheno0-gtex_v8-mashr-Brain_Frontal_Cortex_BA9.csv'),
 PosixPath('/opt/data/results/gls/null_sims/twas/spredixcan/random.pheno0-gtex_v8-mashr-Kidney_Cortex.csv'),
 PosixPath('/opt/data/results/gls/null_sims/twas/spredixcan/random.pheno0-gtex_v8-mashr-Brain_Substantia_nigra.csv')]

In [33]:
# look at the structure of one result
pd.read_csv(spredixcan_result_files["Whole_Blood"]).head()

,gene,gene_name,zscore,effect_size,pvalue,var_g,pred_perf_r2,pred_perf_pval,pred_perf_qval,n_snps_used,n_snps_in_cov,n_snps_in_model,best_gwas_p,largest_weight
0,ENSG00000131236.16,CAP1,-3.882684,NaN,0.000103,8.369900e-05,NaN,NaN,NaN,1,1,1,0.000103,0.021593
1,ENSG00000130787.13,HIP1R,3.862583,NaN,0.000112,3.039669e-03,NaN,NaN,NaN,2,2,2,0.001681,0.087495
2,ENSG00000103018.16,CYB5B,3.478682,NaN,0.000504,6.455623e-03,NaN,NaN,NaN,1,1,1,0.000504,0.114291
3,ENSG00000117906.13,RCN2,3.431258,NaN,0.000601,5.848182e-07,NaN,NaN,NaN,1,1,1,0.000601,0.001696
4,ENSG00000135469.13,COQ10A,3.363631,NaN,0.000769,4.312235e-04,NaN,NaN,NaN,2,2,2,0.013565,0.064460


In [34]:
assert all(f.exists() for f in spredixcan_result_files.values())

In [35]:
spredixcan_dfs = [
    pd.read_csv(
        f, usecols=["gene", "zscore", "pvalue", "n_snps_used", "n_snps_in_model"]
    )
    .dropna(subset=["gene", "zscore", "pvalue"])
    .assign(tissue=t)
    for t, f in spredixcan_result_files.items()
]

In [36]:
assert len(spredixcan_dfs) == len(prediction_model_tissues)

In [37]:
spredixcan_dfs = pd.concat(spredixcan_dfs)

In [38]:
assert spredixcan_dfs["tissue"].unique().shape[0] == len(prediction_model_tissues)

In [39]:
spredixcan_dfs.shape

(653140, 6)

In [40]:
spredixcan_dfs = spredixcan_dfs.assign(
    gene_id=spredixcan_dfs["gene"].apply(lambda g: g.split(".")[0])
)

In [41]:
spredixcan_dfs.head()

,gene,zscore,pvalue,n_snps_used,n_snps_in_model,tissue,gene_id
0,ENSG00000131236.16,-3.882684,0.000103,1,1,Skin_Not_Sun_Exposed_Suprapubic,ENSG00000131236
1,ENSG00000196172.9,-3.710444,0.000207,1,1,Skin_Not_Sun_Exposed_Suprapubic,ENSG00000196172
2,ENSG00000280789.1,3.403263,0.000666,2,2,Skin_Not_Sun_Exposed_Suprapubic,ENSG00000280789
3,ENSG00000215218.3,3.385093,0.000712,3,3,Skin_Not_Sun_Exposed_Suprapubic,ENSG00000215218
4,ENSG00000167371.17,3.381834,0.000720,2,2,Skin_Not_Sun_Exposed_Suprapubic,ENSG00000167371


### Count number of tissues available per gene

In [42]:
spredixcan_genes_n_models = spredixcan_dfs.groupby("gene_id")["tissue"].nunique()

In [43]:
spredixcan_genes_n_models

gene_id
ENSG00000000419     2
ENSG00000000457    48
ENSG00000000460    39
ENSG00000000938    36
ENSG00000000971    34
                   ..
ENSG00000284430     6
ENSG00000284452     1
ENSG00000284513     1
ENSG00000284526    42
ENSG00000284552     1
Name: tissue, Length: 22314, dtype: int64

### Get tissues available per gene

In [44]:
spredixcan_genes_models = spredixcan_dfs.groupby("gene_id")["tissue"].apply(
    lambda x: frozenset(x.tolist())
)

In [45]:
spredixcan_genes_models

gene_id
ENSG00000000419         (Brain_Substantia_nigra, Brain_Hypothalamus)
ENSG00000000457    (Uterus, Muscle_Skeletal, Brain_Cerebellum, Sp...
ENSG00000000460    (Muscle_Skeletal, Brain_Cerebellum, Esophagus_...
ENSG00000000938    (Muscle_Skeletal, Brain_Cerebellum, Spleen, Es...
ENSG00000000971    (Uterus, Brain_Cerebellum, Esophagus_Musculari...
                                         ...                        
ENSG00000284430    (Nerve_Tibial, Uterus, Thyroid, Ovary, Vagina,...
ENSG00000284452                                             (Testis)
ENSG00000284513                        (Brain_Cerebellar_Hemisphere)
ENSG00000284526    (Uterus, Muscle_Skeletal, Spleen, Esophagus_Mu...
ENSG00000284552                                             (Spleen)
Name: tissue, Length: 22314, dtype: object

In [46]:
assert spredixcan_genes_n_models.shape[0] == spredixcan_genes_models.shape[0]

In [47]:
assert spredixcan_genes_n_models.index.equals(spredixcan_genes_models.index)

In [48]:
assert (spredixcan_genes_models.apply(len) <= len(prediction_model_tissues)).all()

In [49]:
spredixcan_genes_models.apply(len).describe()

count    22314.000000
mean        29.270413
std         17.208404
min          1.000000
25%         13.000000
50%         35.000000
75%         45.000000
max         49.000000
Name: tissue, dtype: float64

In [50]:
# testing
assert (
    spredixcan_genes_models.loc[spredixcan_genes_n_models.index]
    .apply(len)
    .equals(spredixcan_genes_n_models)
)

### Add gene name and set index

In [51]:
spredixcan_genes_models = spredixcan_genes_models.to_frame().reset_index()

In [52]:
spredixcan_genes_models.head()

,gene_id,tissue
0,ENSG00000000419,"(Brain_Substantia_nigra, Brain_Hypothalamus)"
1,ENSG00000000457,"(Uterus, Muscle_Skeletal, Brain_Cerebellum, Sp..."
2,ENSG00000000460,"(Muscle_Skeletal, Brain_Cerebellum, Esophagus_..."
3,ENSG00000000938,"(Muscle_Skeletal, Brain_Cerebellum, Spleen, Es..."
4,ENSG00000000971,"(Uterus, Brain_Cerebellum, Esophagus_Musculari..."


In [53]:
spredixcan_genes_models = spredixcan_genes_models.assign(
    gene_name=spredixcan_genes_models["gene_id"].apply(
        lambda g: Gene.GENE_ID_TO_NAME_MAP[g]
    )
)

In [54]:
spredixcan_genes_models = spredixcan_genes_models[["gene_id", "gene_name", "tissue"]]

In [55]:
spredixcan_genes_models = spredixcan_genes_models.set_index("gene_id")

In [56]:
spredixcan_genes_models.head()

,gene_name,tissue
gene_id,,
ENSG00000000419,DPM1,"(Brain_Substantia_nigra, Brain_Hypothalamus)"
ENSG00000000457,SCYL3,"(Uterus, Muscle_Skeletal, Brain_Cerebellum, Sp..."
ENSG00000000460,C1orf112,"(Muscle_Skeletal, Brain_Cerebellum, Esophagus_..."
ENSG00000000938,FGR,"(Muscle_Skeletal, Brain_Cerebellum, Spleen, Es..."
ENSG00000000971,CFH,"(Uterus, Brain_Cerebellum, Esophagus_Musculari..."


### Add number of tissues

In [57]:
spredixcan_genes_models = spredixcan_genes_models.assign(
    n_tissues=spredixcan_genes_models["tissue"].apply(len)
)

In [58]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues
gene_id,,,
ENSG00000000419,DPM1,"(Brain_Substantia_nigra, Brain_Hypothalamus)",2
ENSG00000000457,SCYL3,"(Uterus, Muscle_Skeletal, Brain_Cerebellum, Sp...",48
ENSG00000000460,C1orf112,"(Muscle_Skeletal, Brain_Cerebellum, Esophagus_...",39
ENSG00000000938,FGR,"(Muscle_Skeletal, Brain_Cerebellum, Spleen, Es...",36
ENSG00000000971,CFH,"(Uterus, Brain_Cerebellum, Esophagus_Musculari...",34


### Count number of SNPs predictors used across tissue models

In [59]:
spredixcan_genes_sum_of_n_snps_used = (
    spredixcan_dfs.groupby("gene_id")["n_snps_used"].sum().rename("n_snps_used_sum")
)

In [60]:
spredixcan_genes_sum_of_n_snps_used

gene_id
ENSG00000000419     2
ENSG00000000457    88
ENSG00000000460    62
ENSG00000000938    40
ENSG00000000971    44
                   ..
ENSG00000284430     7
ENSG00000284452     2
ENSG00000284513     1
ENSG00000284526    75
ENSG00000284552     1
Name: n_snps_used_sum, Length: 22314, dtype: int64

In [61]:
# add sum of snps used to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_sum_of_n_snps_used
)

In [62]:
spredixcan_genes_models.shape

(22314, 4)

In [63]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,n_snps_used_sum
gene_id,,,,
ENSG00000000419,DPM1,"(Brain_Substantia_nigra, Brain_Hypothalamus)",2,2
ENSG00000000457,SCYL3,"(Uterus, Muscle_Skeletal, Brain_Cerebellum, Sp...",48,88
ENSG00000000460,C1orf112,"(Muscle_Skeletal, Brain_Cerebellum, Esophagus_...",39,62
ENSG00000000938,FGR,"(Muscle_Skeletal, Brain_Cerebellum, Spleen, Es...",36,40
ENSG00000000971,CFH,"(Uterus, Brain_Cerebellum, Esophagus_Musculari...",34,44


### Count number of SNPs predictors in models across tissue models

In [64]:
spredixcan_genes_sum_of_n_snps_in_model = (
    spredixcan_dfs.groupby("gene_id")["n_snps_in_model"]
    .sum()
    .rename("n_snps_in_model_sum")
)

In [65]:
spredixcan_genes_sum_of_n_snps_in_model

gene_id
ENSG00000000419     2
ENSG00000000457    90
ENSG00000000460    64
ENSG00000000938    40
ENSG00000000971    44
                   ..
ENSG00000284430     7
ENSG00000284452     2
ENSG00000284513     1
ENSG00000284526    79
ENSG00000284552     1
Name: n_snps_in_model_sum, Length: 22314, dtype: int64

In [66]:
# add sum of snps in model to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_sum_of_n_snps_in_model
)

In [67]:
spredixcan_genes_models.shape

(22314, 5)

In [68]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,n_snps_used_sum,n_snps_in_model_sum
gene_id,,,,,
ENSG00000000419,DPM1,"(Brain_Substantia_nigra, Brain_Hypothalamus)",2,2,2
ENSG00000000457,SCYL3,"(Uterus, Muscle_Skeletal, Brain_Cerebellum, Sp...",48,88,90
ENSG00000000460,C1orf112,"(Muscle_Skeletal, Brain_Cerebellum, Esophagus_...",39,62,64
ENSG00000000938,FGR,"(Muscle_Skeletal, Brain_Cerebellum, Spleen, Es...",36,40,40
ENSG00000000971,CFH,"(Uterus, Brain_Cerebellum, Esophagus_Musculari...",34,44,44


### Count number of _unique_ SNPs predictors used and available across tissue models

In [69]:
spredixcan_gene_obj = {
    gene_id: Gene(ensembl_id=gene_id) for gene_id in spredixcan_genes_models.index
}

In [70]:
len(spredixcan_gene_obj)

22314

In [71]:
def _count_unique_snps(gene_id):
    """
    For a gene_id, it counts unique SNPs in all models and their intersection with GWAS SNPs (therefore, used by S-PrediXcan).
    """
    gene_obj = spredixcan_gene_obj[gene_id]
    gene_tissues = spredixcan_genes_models.loc[gene_id, "tissue"]

    gene_unique_snps = set()
    for t in gene_tissues:
        t_snps = set(
            gene_obj.get_prediction_weights(tissue=t, model_type=EQTL_MODEL).index
        )
        gene_unique_snps.update(t_snps)

    gene_unique_snps_in_gwas = gwas_variants_ids_set.intersection(gene_unique_snps)

    return pd.Series(
        {
            "unique_n_snps_in_model": len(gene_unique_snps),
            "unique_n_snps_used": len(gene_unique_snps_in_gwas),
        }
    )

In [72]:
# testing
spredixcan_genes_models[spredixcan_genes_models["n_snps_used_sum"] == 2].head()

,gene_name,tissue,n_tissues,n_snps_used_sum,n_snps_in_model_sum
gene_id,,,,,
ENSG00000000419,DPM1,"(Brain_Substantia_nigra, Brain_Hypothalamus)",2,2,2
ENSG00000007001,UPP2,"(Brain_Nucleus_accumbens_basal_ganglia, Brain_...",2,2,2
ENSG00000007202,KIAA0100,"(Adipose_Subcutaneous, Esophagus_Mucosa)",2,2,2
ENSG00000010256,UQCRC1,"(Whole_Blood, Thyroid)",2,2,2
ENSG00000011451,WIZ,"(Brain_Anterior_cingulate_cortex_BA24, Skin_Su...",2,2,2


In [73]:
# case with two snps, not repeated across tissues
_gene_id = "ENSG00000000419"
display(
    spredixcan_gene_obj[_gene_id].get_prediction_weights(
        tissue="Brain_Hypothalamus", model_type=EQTL_MODEL
    )
)
display(
    spredixcan_gene_obj[_gene_id].get_prediction_weights(
        tissue="Brain_Substantia_nigra", model_type=EQTL_MODEL
    )
)

varID
chr20_50862947_C_T_b38    0.431375
Name: weight, dtype: float64

varID
chr20_50957480_C_T_b38   -0.146796
Name: weight, dtype: float64

In [74]:
_tmp = _count_unique_snps(_gene_id)
assert _tmp.shape[0] == 2
assert _tmp["unique_n_snps_in_model"] == 2
assert _tmp["unique_n_snps_used"] == 2

In [75]:
# case with two snps, repeated across tissues
_gene_id = "ENSG00000007202"
display(
    spredixcan_gene_obj[_gene_id].get_prediction_weights(
        tissue="Adipose_Subcutaneous", model_type=EQTL_MODEL
    )
)
display(
    spredixcan_gene_obj[_gene_id].get_prediction_weights(
        tissue="Esophagus_Mucosa", model_type=EQTL_MODEL
    )
)

varID
chr17_28638994_C_T_b38    0.057102
Name: weight, dtype: float64

varID
chr17_28638994_C_T_b38   -0.159508
Name: weight, dtype: float64

In [76]:
"chr17_28638994_C_T_b38" in gwas_variants_ids_set

True

In [77]:
_tmp = _count_unique_snps(_gene_id)
assert _tmp.shape[0] == 2
assert _tmp["unique_n_snps_in_model"] == 1
assert _tmp["unique_n_snps_used"] == 1

In [78]:
# get unique snps for all genes
spredixcan_genes_unique_n_snps = spredixcan_genes_models.groupby("gene_id").apply(
    lambda x: _count_unique_snps(x.name)
)

In [79]:
spredixcan_genes_unique_n_snps.head()

,unique_n_snps_in_model,unique_n_snps_used
gene_id,,
ENSG00000000419,2,2
ENSG00000000457,18,17
ENSG00000000460,24,22
ENSG00000000938,5,5
ENSG00000000971,12,12


In [80]:
assert (
    spredixcan_genes_unique_n_snps["unique_n_snps_in_model"]
    >= spredixcan_genes_unique_n_snps["unique_n_snps_used"]
).all()

In [81]:
# add unique snps to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(spredixcan_genes_unique_n_snps)

In [82]:
spredixcan_genes_models.shape

(22314, 7)

In [83]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,n_snps_used_sum,n_snps_in_model_sum,unique_n_snps_in_model,unique_n_snps_used
gene_id,,,,,,,
ENSG00000000419,DPM1,"(Brain_Substantia_nigra, Brain_Hypothalamus)",2,2,2,2,2
ENSG00000000457,SCYL3,"(Uterus, Muscle_Skeletal, Brain_Cerebellum, Sp...",48,88,90,18,17
ENSG00000000460,C1orf112,"(Muscle_Skeletal, Brain_Cerebellum, Esophagus_...",39,62,64,24,22
ENSG00000000938,FGR,"(Muscle_Skeletal, Brain_Cerebellum, Spleen, Es...",36,40,40,5,5
ENSG00000000971,CFH,"(Uterus, Brain_Cerebellum, Esophagus_Musculari...",34,44,44,12,12


### Save

In [84]:
spredixcan_genes_models.to_pickle(OUTPUT_DIR_BASE / "gene_tissues.pkl")

## S-MultiXcan results

In [85]:
# TODO: something that could be interesting to do is to compare `n_indep` with the number of independent components I get
smultixcan_results = pd.read_csv(
    SMULTIXCAN_FILE, sep="\t", usecols=["gene", "gene_name", "pvalue", "n"]
)

In [86]:
smultixcan_results.shape

(22317, 4)

In [87]:
smultixcan_results = smultixcan_results.dropna()

In [88]:
smultixcan_results.shape

(22314, 4)

In [89]:
smultixcan_results = smultixcan_results.assign(
    gene_id=smultixcan_results["gene"].apply(lambda g: g.split(".")[0])
)

In [90]:
smultixcan_results.head()

,gene,gene_name,pvalue,n,gene_id
0,ENSG00000131941.7,RHPN2,0.000040,48.0,ENSG00000131941
1,ENSG00000076650.6,GPATCH1,0.000078,40.0,ENSG00000076650
2,ENSG00000100906.10,NFKBIA,0.000096,1.0,ENSG00000100906
3,ENSG00000136319.11,TTC5,0.000109,47.0,ENSG00000136319
4,ENSG00000152990.13,ADGRA3,0.000135,41.0,ENSG00000152990


In [91]:
assert smultixcan_results["gene_id"].is_unique

In [92]:
# testing
_tmp_smultixcan_results_n_models = (
    smultixcan_results.set_index("gene_id")["n"].astype(int).rename("tissue")
)

assert spredixcan_genes_n_models.shape[0] == _tmp_smultixcan_results_n_models.shape[0]
assert spredixcan_genes_n_models.equals(
    _tmp_smultixcan_results_n_models.loc[spredixcan_genes_n_models.index]
)

### Remove duplicated gene names

In [93]:
smultixcan_results["gene_name"].is_unique

False

In [94]:
# list duplicated gene names
_smultixcan_duplicated_gene_names = smultixcan_results[
    smultixcan_results["gene_name"].duplicated(keep=False)
]
display(_smultixcan_duplicated_gene_names)

,gene,gene_name,pvalue,n,gene_id
1461,ENSG00000235641.4,LINC00484,0.068450,35.0,ENSG00000235641
2390,ENSG00000237667.5,LINC01115,0.112329,22.0,ENSG00000237667
5198,ENSG00000272342.1,LINC01115,0.243122,4.0,ENSG00000272342
5883,ENSG00000229694.6,LINC00484,0.274814,40.0,ENSG00000229694
6129,ENSG00000147676.13,MAL2,0.286003,29.0,ENSG00000147676
6207,ENSG00000283992.1,LYNX1,0.289628,45.0,ENSG00000283992
10559,ENSG00000180155.19,LYNX1,0.486960,48.0,ENSG00000180155
14026,ENSG00000235271.5,LINC01422,0.643355,38.0,ENSG00000235271
14251,ENSG00000253972.5,MAL2,0.654873,6.0,ENSG00000253972
20086,ENSG00000182957.15,SPATA13,0.906729,40.0,ENSG00000182957


In [95]:
# TODO: my strategy below to handle duplicated gene names is to keep the first one
#  it might be better to have another strategy, maybe keeping the most significant

In [96]:
smultixcan_results = smultixcan_results.drop_duplicates(
    subset=["gene_name"], keep="first"
)
display(smultixcan_results.shape)

(22308, 5)

### Get common genes with MultiPLIER

In [97]:
common_genes = set(multiplier_z_genes).intersection(
    set(smultixcan_results["gene_name"])
)

In [98]:
len(common_genes)

6444

In [99]:
sorted(list(common_genes))[:5]

['A2M', 'AAAS', 'AANAT', 'AARS', 'AARS2']

In [100]:
assert smultixcan_results[smultixcan_results["gene_name"].isin(common_genes)].shape[
    0
] == len(common_genes)

### Save

In [101]:
with open(OUTPUT_DIR_BASE / "common_genes.pkl", "wb") as handle:
    pickle.dump(common_genes, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Get gene objects

In [102]:
multiplier_gene_obj = {
    gene_name: Gene(name=gene_name)
    for gene_name in common_genes
    if gene_name in Gene.GENE_NAME_TO_ID_MAP
}

In [103]:
len(multiplier_gene_obj)

6444

In [104]:
assert multiplier_gene_obj["GAS6"].ensembl_id == "ENSG00000183087"

In [105]:
_gene_obj = list(multiplier_gene_obj.values())

genes_info = pd.DataFrame(
    {
        "name": [g.name for g in _gene_obj],
        "id": [g.ensembl_id for g in _gene_obj],
        "chr": [g.chromosome for g in _gene_obj],
        "band": [g.band for g in _gene_obj],
        "start_position": [g.get_attribute("start_position") for g in _gene_obj],
        "end_position": [g.get_attribute("end_position") for g in _gene_obj],
    }
)

In [106]:
genes_info = genes_info.assign(
    gene_length=genes_info.apply(
        lambda x: x["end_position"] - x["start_position"], axis=1
    )
)

In [107]:
genes_info.shape

(6444, 7)

In [108]:
genes_info.head()

,name,id,chr,band,start_position,end_position,gene_length
0,PDGFRB,ENSG00000113721,5,5q32,150113839.0,150155872.0,42033.0
1,ASB1,ENSG00000065802,2,2q37.3,238426742.0,238452250.0,25508.0
2,RPS8,ENSG00000142937,1,1p34.1,44775251.0,44778779.0,3528.0
3,ZNF473,ENSG00000142528,19,19q13.33,50025714.0,50053414.0,27700.0
4,FDPS,ENSG00000160752,1,1q22,155308748.0,155320666.0,11918.0


## Save

In [109]:
genes_info.to_pickle(OUTPUT_DIR_BASE / "genes_info.pkl")